In [4]:
import pandas as pd
import yaml

In [35]:
config = yaml.safe_load(open('../../configs/config.yml'))

In [33]:
def get_data(config=config):

    # get paths
    paths = config.get('paths', 'NO "PATHS" SET IN CONFIG')

    # read
    x_train = pd.read_csv("../../" + paths.get('train_values'))
    y_train = pd.read_csv("../../" + paths.get('train_labels'))

    # join
    df_train = pd.merge(x_train, y_train, on="building_id")

    return df_train

In [34]:
%%time
df_train = get_data(config)

# Column Types
---

In [ ]:
df_train.info()

In [ ]:
def format_dtypes(df):
    # categorical values
    cat_cols = df.select_dtypes(include='object').columns.tolist()
    df[cat_cols] = df[cat_cols].astype('category')

    return df

In [ ]:
df_train = format_dtypes(train)

In [ ]:
df_train.info()

# Outliers
---
To handle outliers the statistical IQR Method is used. Outliers which are filtered by this method are either replaced with the upper or the lower bound value, another option would be to simply drop them.

In [ ]:
def find_outliers(df, replace=True, drop=False):
    return df

# Summary

In [40]:
def clean_data(df, config):
    # Drop unnecessary columns
    drop_cols = config.get("data_cleaning", "NO DATA CLEANING DEFINED").get("columns_to_remove")
    df = df.drop(drop_cols, axis=1)
    # Format column types
    df = format_dtypes(df)
    # Handle outliers
    df = find_outliers(df)

    return df

In [41]:
%%time
cl_df_train = clean_data(df_train, config=config)
cl_x_train = cl_df_train.drop("damage_grade", axis=1)
cl_y_train = cl_df_train["damage_grade"]

KeyError: "['', '', ''] not found in axis"